# Prediction for Survival of the Titanic desaster with Catboost Model

## 1. Importing the libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score
from catboost import CatBoostClassifier

## 2. Reading the datasets 

In [2]:
train = pd.read_csv('../00_data/train.csv')
test = pd.read_csv('../00_data/test.csv')

## 3. Exploring the datasets

### 3.1 Training set

In [3]:
print(train.head())

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  


In [4]:
print(train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


In [5]:
print(train.isna().sum())

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


### 3.2 Test set

In [6]:
print(test.head())

   PassengerId  Pclass                                          Name     Sex  \
0          892       3                              Kelly, Mr. James    male   
1          893       3              Wilkes, Mrs. James (Ellen Needs)  female   
2          894       2                     Myles, Mr. Thomas Francis    male   
3          895       3                              Wirz, Mr. Albert    male   
4          896       3  Hirvonen, Mrs. Alexander (Helga E Lindqvist)  female   

    Age  SibSp  Parch   Ticket     Fare Cabin Embarked  
0  34.5      0      0   330911   7.8292   NaN        Q  
1  47.0      1      0   363272   7.0000   NaN        S  
2  62.0      0      0   240276   9.6875   NaN        Q  
3  27.0      0      0   315154   8.6625   NaN        S  
4  22.0      1      1  3101298  12.2875   NaN        S  


In [7]:
print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB
None


In [8]:
print(test.isna().sum())

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


## 4. Preprocessing

### 4.1 Drop unnecessary columns

#### 4.1.1 Training set

In [9]:
print(train['Ticket'])
print(train['Ticket'].nunique())

0             A/5 21171
1              PC 17599
2      STON/O2. 3101282
3                113803
4                373450
             ...       
886              211536
887              112053
888          W./C. 6607
889              111369
890              370376
Name: Ticket, Length: 891, dtype: object
681


Drop PassengerId, Name, Ticket and Embarked as these features don't supply useful information for the prediction of survival.
Drop Cabin, too, as there are too many missing values.

In [10]:
train.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin', 'Embarked'], inplace=True)

#### 4.1.2 Test set

For the test set we keep the PassengerId, as we need it for the construction of the result dataframe containing the predictions for survival of a passenger made by the model.

In [11]:
test.drop(columns=['Name', 'Ticket', 'Cabin', 'Embarked'], inplace=True)

### 4.2 Splitting the Training set in train and test data

In [12]:
train_train, train_test = train_test_split(train, test_size=0.2, random_state=0)

### 4.3 Creating X and y for model development

In [13]:
train_train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
140,0,3,female,NaN,0,2,15.2458
439,0,2,male,31.0,0,0,10.5000
817,0,2,male,31.0,1,1,37.0042
378,0,3,male,20.0,0,0,4.0125
491,0,3,male,21.0,0,0,7.2500


In [14]:
X_train = train_train.iloc[:, 1:].values
y_train = train_train.iloc[:, 0].values

In [15]:
print(X_train)
print(y_train)

[[3 'female' nan 0 2 15.2458]
 [2 'male' 31.0 0 0 10.5]
 [2 'male' 31.0 1 1 37.0042]
 ...
 [3 'male' nan 0 0 7.7333]
 [3 'female' 36.0 1 0 17.4]
 [2 'male' 60.0 1 1 39.0]]
[0 0 0 0 0 0 0 1 0 0 1 0 0 1 1 0 0 0 1 1 0 1 0 0 1 1 0 0 0 1 0 0 1 0 0 0 0
 0 1 0 1 0 1 1 0 1 0 0 1 1 0 0 1 1 0 0 0 1 1 1 0 0 1 0 0 1 1 0 0 0 1 0 0 0
 0 0 0 1 1 1 0 0 0 0 0 0 0 1 0 1 1 0 0 0 1 0 0 0 1 1 0 0 1 0 1 0 0 0 0 0 0
 1 0 1 1 1 0 0 0 1 0 1 0 0 1 1 1 0 0 1 0 1 0 0 0 0 1 0 1 0 1 0 1 1 1 0 0 0
 0 0 0 1 0 1 0 0 0 1 0 1 0 0 0 1 0 1 1 1 0 0 0 1 1 0 0 1 0 1 0 0 0 0 1 1 1
 0 1 0 1 0 1 0 0 1 0 0 0 0 0 1 0 1 0 1 0 0 0 0 0 1 0 0 1 0 0 0 1 0 0 1 0 0
 1 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 0 1 1 0 1 0 1
 0 0 0 1 1 0 0 0 0 0 0 1 0 0 1 1 0 0 0 1 0 1 0 0 1 0 0 1 1 0 0 1 1 1 0 1 0
 0 1 0 0 0 0 0 1 0 0 0 1 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 1 0 0
 1 0 1 0 1 0 0 0 1 1 1 1 0 0 0 0 0 0 0 1 1 1 0 0 0 1 0 1 1 1 0 0 0 0 1 1 0
 0 1 0 1 1 0 0 0 0 1 1 0 1 0 1 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 1
 1 

In [16]:
X_test = train_test.iloc[:, 1:].values
y_test = train_test.iloc[:, 0].values

In [17]:
print(X_test)
print(y_test)

[[3 'male' nan 0 0 14.4583]
 [3 'male' nan 0 0 7.55]
 [3 'male' 7.0 4 1 29.125]
 ...
 [1 'female' 31.0 1 0 113.275]
 [3 'male' 23.0 0 0 7.8542]
 [3 'male' 19.0 0 0 8.05]]
[0 0 0 1 1 1 1 1 1 1 0 1 0 1 1 0 0 0 0 1 0 1 0 0 0 1 0 1 1 0 0 1 0 1 0 1 0
 0 0 0 1 0 0 0 1 0 0 1 0 0 1 1 1 0 1 0 0 0 0 1 0 0 1 0 1 0 1 0 1 1 1 1 0 0
 0 1 0 0 0 0 0 1 0 0 0 1 1 1 1 0 0 0 1 1 0 0 1 0 0 1 0 0 0 0 0 1 1 0 0 1 0
 1 1 0 1 1 1 1 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1
 1 0 0 1 0 0 1 0 0 1 0 1 0 1 1 1 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0]


### 4.4 Creating X as input for prediction

In [18]:
print(test.head())

   PassengerId  Pclass     Sex   Age  SibSp  Parch     Fare
0          892       3    male  34.5      0      0   7.8292
1          893       3  female  47.0      1      0   7.0000
2          894       2    male  62.0      0      0   9.6875
3          895       3    male  27.0      0      0   8.6625
4          896       3  female  22.0      1      1  12.2875


In [19]:
X_pred = test.iloc[:, 1:].values

In [20]:
print(X_pred)

[[3 'male' 34.5 0 0 7.8292]
 [3 'female' 47.0 1 0 7.0]
 [2 'male' 62.0 0 0 9.6875]
 ...
 [3 'male' 38.5 0 0 7.25]
 [3 'male' nan 0 0 8.05]
 [3 'male' nan 1 1 22.3583]]


### 4.5 Handle missing data 

#### 4.5.1 Training set (model development)

In [21]:
print(train_train.isna().sum())

Survived      0
Pclass        0
Sex           0
Age         141
SibSp         0
Parch         0
Fare          0
dtype: int64


##### Calculate treshold for missing values

In [22]:
treshold = len(train_train) * 0.05
print(treshold)
print(len(train_train))

35.6
712


As it is already obvious, the number of missing values in Age is far above the treshold so we cannot simply drop them. 

We have now two options: 
- drop feature Age as there are many missing values in it
- handle missing data by imputing mean e.g.

First we try building the model with feature Age and imputed values. 

##### Keep Age and impute missing values

In [23]:
imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imputer.fit(np.reshape(X_train[:, 2], (len(X_train[:, 2]), 1)))
X_train[:, 2] = imputer.transform(np.reshape(X_train[:, 2], (len(X_train[:, 2]), 1))).reshape(-1)

In [24]:
print(X_train)

[[3 'female' 24.0 0 2 15.2458]
 [2 'male' 31.0 0 0 10.5]
 [2 'male' 31.0 1 1 37.0042]
 ...
 [3 'male' 24.0 0 0 7.7333]
 [3 'female' 36.0 1 0 17.4]
 [2 'male' 60.0 1 1 39.0]]


#### 4.5.2 Test set (model development)

In [25]:
print(train_test.isna().sum())

Survived     0
Pclass       0
Sex          0
Age         36
SibSp        0
Parch        0
Fare         0
dtype: int64


In [26]:
X_test[:, 2] = imputer.transform(np.reshape(X_test[:, 2], (len(X_test[:, 2]), 1))).reshape(-1)

In [27]:
print(X_test)

[[3 'male' 24.0 0 0 14.4583]
 [3 'male' 24.0 0 0 7.55]
 [3 'male' 7.0 4 1 29.125]
 ...
 [1 'female' 31.0 1 0 113.275]
 [3 'male' 23.0 0 0 7.8542]
 [3 'male' 19.0 0 0 8.05]]


#### 4.5.3 Test set (test set to predict)

In [28]:
print(test.isna().sum())

PassengerId     0
Pclass          0
Sex             0
Age            86
SibSp           0
Parch           0
Fare            1
dtype: int64


Here we have one additional column with missing values in Fare.

##### Calculate treshold for missing values

In [29]:
treshold = len(test) * 0.05
print(treshold)
print(len(test))

20.900000000000002
418


The missing value in Fare is below the treshold and the observation could be dropped. But as this observation belongs to the test set to predict, we have to deal with it and impute the missing value instead. The missing values in Age are filled by the imputer used for the training set.

##### Impute missing values for Age

In [30]:
X_pred[:, 2] = imputer.transform(np.reshape(X_pred[:, 2], (len(X_pred[:, 2]), 1))).reshape(-1)

In [31]:
print(X_pred)

[[3 'male' 34.5 0 0 7.8292]
 [3 'female' 47.0 1 0 7.0]
 [2 'male' 62.0 0 0 9.6875]
 ...
 [3 'male' 38.5 0 0 7.25]
 [3 'male' 24.0 0 0 8.05]
 [3 'male' 24.0 1 1 22.3583]]


In [32]:
##### Impute missing values for Fare

In [33]:
imputer_fare = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imputer_fare.fit(np.reshape(X_pred[:, 5], (len(X_pred[:, 5]), 1)))
X_pred[:, 5] = imputer.transform(np.reshape(X_pred[:, 5], (len(X_pred[:, 5]), 1))).reshape(-1)

### 4.6 Encode categorical values

#### 4.6.1 Training set (model development)

In [34]:
print(X_train)

[[3 'female' 24.0 0 2 15.2458]
 [2 'male' 31.0 0 0 10.5]
 [2 'male' 31.0 1 1 37.0042]
 ...
 [3 'male' 24.0 0 0 7.7333]
 [3 'female' 36.0 1 0 17.4]
 [2 'male' 60.0 1 1 39.0]]


Encode values in PClass (col 0) and Sex (col 1).

In [35]:
ct_X = ColumnTransformer(transformers=[('encoder', OneHotEncoder(handle_unknown='ignore'), [0, 1])], remainder='passthrough')
ct_X.fit(X_train)
X_train = ct_X.transform(X_train)

In [36]:
print(X_train)

[[0.0 0.0 1.0 ... 0 2 15.2458]
 [0.0 1.0 0.0 ... 0 0 10.5]
 [0.0 1.0 0.0 ... 1 1 37.0042]
 ...
 [0.0 0.0 1.0 ... 0 0 7.7333]
 [0.0 0.0 1.0 ... 1 0 17.4]
 [0.0 1.0 0.0 ... 1 1 39.0]]


#### 4.6.2 Test set (model input)

In [37]:
X_test = ct_X.transform(X_test)

In [38]:
print(X_test)

[[0.0 0.0 1.0 ... 0 0 14.4583]
 [0.0 0.0 1.0 ... 0 0 7.55]
 [0.0 0.0 1.0 ... 4 1 29.125]
 ...
 [1.0 0.0 0.0 ... 1 0 113.275]
 [0.0 0.0 1.0 ... 0 0 7.8542]
 [0.0 0.0 1.0 ... 0 0 8.05]]


#### 4.6.3 Test set (test set to predict)

In [39]:
X_pred = ct_X.transform(X_pred)

In [40]:
print(X_pred)

[[0.0 0.0 1.0 ... 0 0 7.8292]
 [0.0 0.0 1.0 ... 1 0 7.0]
 [0.0 1.0 0.0 ... 0 0 9.6875]
 ...
 [0.0 0.0 1.0 ... 0 0 7.25]
 [0.0 0.0 1.0 ... 0 0 8.05]
 [0.0 0.0 1.0 ... 1 1 22.3583]]


### 4.7 Feature Scaling

#### 4.7.1 Training set (model development)

In [41]:
sc = StandardScaler()
X_train[:, 2:] = sc.fit_transform(X_train[:, 2:])

In [42]:
print(X_train)

[[0.0 0.0 0.8932971498777942 ... -0.46445233851359824 1.9592640285249252
  -0.3316790433256366]
 [0.0 1.0 -1.119448327062046 ... -0.46445233851359824 -0.4774101868632787
  -0.4264054204191258]
 [0.0 1.0 -1.119448327062046 ... 0.41270963767123453 0.7409269208308232
  0.10261957931985559]
 ...
 [0.0 0.0 0.8932971498777942 ... -0.46445233851359824 -0.4774101868632787
  -0.4816288743161625]
 [0.0 0.0 0.8932971498777942 ... 0.41270963767123453 -0.4774101868632787
  -0.2886811164145163]
 [0.0 1.0 -1.119448327062046 ... 0.41270963767123453 0.7409269208308232
  0.1424558352520875]]


#### 4.7.2 Test set (model development)

In [43]:
X_test[:, 2:] = sc.transform(X_test[:, 2:])

In [44]:
print(X_test)

[[0.0 0.0 0.8932971498777942 ... -0.46445233851359824 -0.4774101868632787
  -0.3473975780218148]
 [0.0 0.0 0.8932971498777942 ... -0.46445233851359824 -0.4774101868632787
  -0.485287550392111]
 [0.0 0.0 0.8932971498777942 ... 3.0441955662257327 0.7409269208308232
  -0.05464959982697327]
 ...
 [1.0 0.0 -1.119448327062046 ... 0.41270963767123453 -0.4774101868632787
  1.6249881077075041]
 [0.0 0.0 0.8932971498777942 ... -0.46445233851359824 -0.4774101868632787
  -0.4792157049894731]
 [0.0 0.0 0.8932971498777942 ... -0.46445233851359824 -0.4774101868632787
  -0.4753075283627915]]


#### 4.7.3 Test set (test set to predict)

In [45]:
X_pred[:, 2:] = sc.transform(X_pred[:, 2:])

In [46]:
print(X_pred)

[[0.0 0.0 0.8932971498777942 ... -0.46445233851359824 -0.4774101868632787
  -0.479714706090939]
 [0.0 0.0 0.8932971498777942 ... 0.41270963767123453 -0.4774101868632787
  -0.49626557462436255]
 [0.0 1.0 -1.119448327062046 ... -0.46445233851359824 -0.4774101868632787
  -0.44262295621677006]
 ...
 [0.0 0.0 0.8932971498777942 ... -0.46445233851359824 -0.4774101868632787
  -0.49127556360970276]
 [0.0 0.0 0.8932971498777942 ... -0.46445233851359824 -0.4774101868632787
  -0.4753075283627915]
 [0.0 0.0 0.8932971498777942 ... 0.41270963767123453 0.7409269208308232
  -0.1897132299585662]]


### 4.8 Build the CatBoost model

In [47]:
classifier = CatBoostClassifier()
classifier.fit(X_train, y_train)

Learning rate set to 0.008911
0:	learn: 0.6867834	total: 150ms	remaining: 2m 29s
1:	learn: 0.6806759	total: 160ms	remaining: 1m 19s
2:	learn: 0.6759231	total: 169ms	remaining: 56.3s
3:	learn: 0.6704188	total: 176ms	remaining: 43.8s
4:	learn: 0.6660387	total: 180ms	remaining: 35.8s
5:	learn: 0.6610879	total: 185ms	remaining: 30.7s
6:	learn: 0.6558998	total: 190ms	remaining: 27s
7:	learn: 0.6511135	total: 194ms	remaining: 24s
8:	learn: 0.6461282	total: 197ms	remaining: 21.7s
9:	learn: 0.6408987	total: 201ms	remaining: 19.9s
10:	learn: 0.6366610	total: 203ms	remaining: 18.3s
11:	learn: 0.6327431	total: 205ms	remaining: 16.9s
12:	learn: 0.6287332	total: 207ms	remaining: 15.7s
13:	learn: 0.6247303	total: 209ms	remaining: 14.8s
14:	learn: 0.6212176	total: 211ms	remaining: 13.8s
15:	learn: 0.6165896	total: 214ms	remaining: 13.1s
16:	learn: 0.6124960	total: 216ms	remaining: 12.5s
17:	learn: 0.6081248	total: 218ms	remaining: 11.9s
18:	learn: 0.6038448	total: 220ms	remaining: 11.4s
19:	learn: 0.

208:	learn: 0.3900579	total: 663ms	remaining: 2.51s
209:	learn: 0.3897264	total: 665ms	remaining: 2.5s
210:	learn: 0.3895054	total: 668ms	remaining: 2.5s
211:	learn: 0.3894028	total: 670ms	remaining: 2.49s
212:	learn: 0.3891462	total: 672ms	remaining: 2.48s
213:	learn: 0.3887921	total: 674ms	remaining: 2.48s
214:	learn: 0.3886130	total: 677ms	remaining: 2.47s
215:	learn: 0.3884294	total: 679ms	remaining: 2.46s
216:	learn: 0.3882635	total: 681ms	remaining: 2.46s
217:	learn: 0.3880472	total: 684ms	remaining: 2.45s
218:	learn: 0.3876608	total: 686ms	remaining: 2.45s
219:	learn: 0.3871187	total: 689ms	remaining: 2.44s
220:	learn: 0.3866065	total: 691ms	remaining: 2.44s
221:	learn: 0.3864192	total: 694ms	remaining: 2.43s
222:	learn: 0.3862730	total: 696ms	remaining: 2.42s
223:	learn: 0.3860444	total: 698ms	remaining: 2.42s
224:	learn: 0.3858143	total: 701ms	remaining: 2.41s
225:	learn: 0.3855244	total: 703ms	remaining: 2.41s
226:	learn: 0.3853132	total: 705ms	remaining: 2.4s
227:	learn: 0.3

417:	learn: 0.3538874	total: 1.17s	remaining: 1.63s
418:	learn: 0.3538028	total: 1.17s	remaining: 1.63s
419:	learn: 0.3536224	total: 1.18s	remaining: 1.62s
420:	learn: 0.3535221	total: 1.18s	remaining: 1.62s
421:	learn: 0.3533918	total: 1.18s	remaining: 1.62s
422:	learn: 0.3533485	total: 1.18s	remaining: 1.61s
423:	learn: 0.3532373	total: 1.19s	remaining: 1.61s
424:	learn: 0.3532077	total: 1.19s	remaining: 1.61s
425:	learn: 0.3531141	total: 1.19s	remaining: 1.6s
426:	learn: 0.3529968	total: 1.21s	remaining: 1.62s
427:	learn: 0.3528739	total: 1.21s	remaining: 1.61s
428:	learn: 0.3528658	total: 1.21s	remaining: 1.61s
429:	learn: 0.3525159	total: 1.21s	remaining: 1.61s
430:	learn: 0.3522045	total: 1.22s	remaining: 1.6s
431:	learn: 0.3519352	total: 1.22s	remaining: 1.6s
432:	learn: 0.3517724	total: 1.22s	remaining: 1.6s
433:	learn: 0.3517302	total: 1.22s	remaining: 1.59s
434:	learn: 0.3516914	total: 1.23s	remaining: 1.59s
435:	learn: 0.3515629	total: 1.23s	remaining: 1.59s
436:	learn: 0.35

607:	learn: 0.3327955	total: 1.67s	remaining: 1.08s
608:	learn: 0.3327201	total: 1.68s	remaining: 1.07s
609:	learn: 0.3326553	total: 1.68s	remaining: 1.07s
610:	learn: 0.3324339	total: 1.68s	remaining: 1.07s
611:	learn: 0.3323407	total: 1.68s	remaining: 1.07s
612:	learn: 0.3322621	total: 1.69s	remaining: 1.06s
613:	learn: 0.3322052	total: 1.69s	remaining: 1.06s
614:	learn: 0.3321260	total: 1.69s	remaining: 1.06s
615:	learn: 0.3320761	total: 1.69s	remaining: 1.05s
616:	learn: 0.3319851	total: 1.7s	remaining: 1.05s
617:	learn: 0.3319091	total: 1.7s	remaining: 1.05s
618:	learn: 0.3317639	total: 1.7s	remaining: 1.05s
619:	learn: 0.3317042	total: 1.7s	remaining: 1.04s
620:	learn: 0.3314399	total: 1.71s	remaining: 1.04s
621:	learn: 0.3312437	total: 1.71s	remaining: 1.04s
622:	learn: 0.3311651	total: 1.71s	remaining: 1.03s
623:	learn: 0.3309791	total: 1.71s	remaining: 1.03s
624:	learn: 0.3308883	total: 1.72s	remaining: 1.03s
625:	learn: 0.3308477	total: 1.72s	remaining: 1.03s
626:	learn: 0.33

802:	learn: 0.3135191	total: 2.16s	remaining: 530ms
803:	learn: 0.3134476	total: 2.16s	remaining: 527ms
804:	learn: 0.3133592	total: 2.17s	remaining: 525ms
805:	learn: 0.3132311	total: 2.17s	remaining: 522ms
806:	learn: 0.3130701	total: 2.17s	remaining: 519ms
807:	learn: 0.3129699	total: 2.17s	remaining: 516ms
808:	learn: 0.3128638	total: 2.17s	remaining: 513ms
809:	learn: 0.3126994	total: 2.18s	remaining: 511ms
810:	learn: 0.3126947	total: 2.18s	remaining: 508ms
811:	learn: 0.3126477	total: 2.18s	remaining: 505ms
812:	learn: 0.3125805	total: 2.18s	remaining: 502ms
813:	learn: 0.3123865	total: 2.19s	remaining: 499ms
814:	learn: 0.3123438	total: 2.19s	remaining: 497ms
815:	learn: 0.3123031	total: 2.19s	remaining: 494ms
816:	learn: 0.3121712	total: 2.19s	remaining: 491ms
817:	learn: 0.3120992	total: 2.19s	remaining: 488ms
818:	learn: 0.3120501	total: 2.2s	remaining: 486ms
819:	learn: 0.3119796	total: 2.2s	remaining: 483ms
820:	learn: 0.3118527	total: 2.2s	remaining: 480ms
821:	learn: 0.3

965:	learn: 0.2974140	total: 2.54s	remaining: 89.5ms
966:	learn: 0.2973329	total: 2.55s	remaining: 86.9ms
967:	learn: 0.2970516	total: 2.55s	remaining: 84.3ms
968:	learn: 0.2968480	total: 2.55s	remaining: 81.6ms
969:	learn: 0.2968114	total: 2.55s	remaining: 79ms
970:	learn: 0.2967588	total: 2.56s	remaining: 76.3ms
971:	learn: 0.2965991	total: 2.56s	remaining: 73.7ms
972:	learn: 0.2965343	total: 2.56s	remaining: 71ms
973:	learn: 0.2965104	total: 2.56s	remaining: 68.4ms
974:	learn: 0.2964501	total: 2.56s	remaining: 65.8ms
975:	learn: 0.2964041	total: 2.57s	remaining: 63.1ms
976:	learn: 0.2963643	total: 2.57s	remaining: 60.5ms
977:	learn: 0.2963157	total: 2.57s	remaining: 57.8ms
978:	learn: 0.2962694	total: 2.57s	remaining: 55.2ms
979:	learn: 0.2962046	total: 2.58s	remaining: 52.6ms
980:	learn: 0.2961148	total: 2.58s	remaining: 49.9ms
981:	learn: 0.2959945	total: 2.58s	remaining: 47.3ms
982:	learn: 0.2959740	total: 2.58s	remaining: 44.7ms
983:	learn: 0.2958378	total: 2.58s	remaining: 42ms

### 4.9. Predict Test set (model development )

In [48]:
y_model_pred = classifier.predict(X_test)

In [49]:
print(np.concatenate((y_model_pred.reshape(len(y_model_pred),1), y_test.reshape(len(y_test),1)),1))

[[0 0]
 [0 0]
 [0 0]
 [1 1]
 [0 1]
 [0 1]
 [1 1]
 [1 1]
 [0 1]
 [1 1]
 [0 0]
 [1 1]
 [0 0]
 [1 1]
 [1 1]
 [1 0]
 [0 0]
 [0 0]
 [0 0]
 [1 1]
 [0 0]
 [1 1]
 [0 0]
 [0 0]
 [1 0]
 [1 1]
 [0 0]
 [1 1]
 [1 1]
 [0 0]
 [0 0]
 [1 1]
 [0 0]
 [0 1]
 [0 0]
 [0 1]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 1]
 [0 0]
 [0 0]
 [0 0]
 [1 1]
 [0 0]
 [0 0]
 [1 1]
 [0 0]
 [0 0]
 [0 1]
 [0 1]
 [1 1]
 [0 0]
 [0 1]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 1]
 [0 0]
 [0 0]
 [1 1]
 [0 0]
 [1 1]
 [0 0]
 [1 1]
 [0 0]
 [1 1]
 [1 1]
 [1 1]
 [0 1]
 [0 0]
 [0 0]
 [0 0]
 [1 1]
 [1 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [1 1]
 [1 0]
 [0 0]
 [0 0]
 [1 1]
 [1 1]
 [0 1]
 [1 1]
 [0 0]
 [0 0]
 [0 0]
 [1 1]
 [1 1]
 [0 0]
 [0 0]
 [1 1]
 [0 0]
 [0 0]
 [0 1]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 1]
 [1 1]
 [0 0]
 [0 0]
 [1 1]
 [0 0]
 [1 1]
 [1 1]
 [1 0]
 [0 1]
 [1 1]
 [1 1]
 [1 1]
 [0 0]
 [1 1]
 [0 1]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 1]
 [0 0]
 [0 0]
 [1 0]
 [0 0]
 [0 0]
 [1 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [1 1]
 [0 0]

### 4.10. Print Confusion Matrix

In [50]:
print(confusion_matrix(y_test, y_model_pred))

[[100  10]
 [ 20  49]]


In [51]:
print(accuracy_score(y_test, y_model_pred) * 100)

83.24022346368714


### 4.11. Applying k-Fold Cross Validation

In [52]:
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Learning rate set to 0.008515
0:	learn: 0.6889675	total: 4.78ms	remaining: 4.77s
1:	learn: 0.6844953	total: 8.43ms	remaining: 4.21s
2:	learn: 0.6792068	total: 13.2ms	remaining: 4.38s
3:	learn: 0.6740307	total: 17.7ms	remaining: 4.4s
4:	learn: 0.6688075	total: 21ms	remaining: 4.18s
5:	learn: 0.6639394	total: 23.8ms	remaining: 3.95s
6:	learn: 0.6590574	total: 27ms	remaining: 3.83s
7:	learn: 0.6544640	total: 29.9ms	remaining: 3.7s
8:	learn: 0.6495864	total: 32.2ms	remaining: 3.54s
9:	learn: 0.6446331	total: 34.4ms	remaining: 3.41s
10:	learn: 0.6412601	total: 36.2ms	remaining: 3.26s
11:	learn: 0.6374156	total: 38.1ms	remaining: 3.14s
12:	learn: 0.6334537	total: 40.2ms	remaining: 3.05s
13:	learn: 0.6295359	total: 42.7ms	remaining: 3.01s
14:	learn: 0.6261068	total: 44.5ms	remaining: 2.92s
15:	learn: 0.6227472	total: 46ms	remaining: 2.83s
16:	learn: 0.6204538	total: 47.3ms	remaining: 2.73s
17:	learn: 0.6160479	total: 49.5ms	remaining: 2.7s
18:	learn: 0.6120052	total: 51.7ms	remaining: 2.67s
1

183:	learn: 0.4019998	total: 440ms	remaining: 1.95s
184:	learn: 0.4016809	total: 442ms	remaining: 1.95s
185:	learn: 0.4011101	total: 444ms	remaining: 1.94s
186:	learn: 0.4007340	total: 446ms	remaining: 1.94s
187:	learn: 0.4004042	total: 449ms	remaining: 1.94s
188:	learn: 0.4000968	total: 451ms	remaining: 1.93s
189:	learn: 0.3998103	total: 454ms	remaining: 1.93s
190:	learn: 0.3993966	total: 456ms	remaining: 1.93s
191:	learn: 0.3988271	total: 458ms	remaining: 1.93s
192:	learn: 0.3983386	total: 461ms	remaining: 1.93s
193:	learn: 0.3981301	total: 463ms	remaining: 1.92s
194:	learn: 0.3978918	total: 466ms	remaining: 1.92s
195:	learn: 0.3975643	total: 470ms	remaining: 1.93s
196:	learn: 0.3971520	total: 472ms	remaining: 1.92s
197:	learn: 0.3968773	total: 474ms	remaining: 1.92s
198:	learn: 0.3965500	total: 477ms	remaining: 1.92s
199:	learn: 0.3962332	total: 479ms	remaining: 1.92s
200:	learn: 0.3958874	total: 481ms	remaining: 1.91s
201:	learn: 0.3955411	total: 483ms	remaining: 1.91s
202:	learn: 

346:	learn: 0.3640216	total: 810ms	remaining: 1.52s
347:	learn: 0.3638035	total: 813ms	remaining: 1.52s
348:	learn: 0.3637732	total: 814ms	remaining: 1.52s
349:	learn: 0.3636358	total: 817ms	remaining: 1.52s
350:	learn: 0.3634990	total: 819ms	remaining: 1.51s
351:	learn: 0.3630898	total: 821ms	remaining: 1.51s
352:	learn: 0.3630876	total: 823ms	remaining: 1.51s
353:	learn: 0.3627500	total: 825ms	remaining: 1.51s
354:	learn: 0.3624808	total: 828ms	remaining: 1.5s
355:	learn: 0.3624012	total: 830ms	remaining: 1.5s
356:	learn: 0.3621888	total: 832ms	remaining: 1.5s
357:	learn: 0.3620694	total: 835ms	remaining: 1.5s
358:	learn: 0.3619381	total: 837ms	remaining: 1.49s
359:	learn: 0.3618126	total: 840ms	remaining: 1.49s
360:	learn: 0.3616364	total: 842ms	remaining: 1.49s
361:	learn: 0.3615919	total: 845ms	remaining: 1.49s
362:	learn: 0.3614465	total: 847ms	remaining: 1.49s
363:	learn: 0.3613093	total: 849ms	remaining: 1.48s
364:	learn: 0.3611720	total: 851ms	remaining: 1.48s
365:	learn: 0.36

506:	learn: 0.3433293	total: 1.18s	remaining: 1.15s
507:	learn: 0.3432050	total: 1.18s	remaining: 1.14s
508:	learn: 0.3430403	total: 1.18s	remaining: 1.14s
509:	learn: 0.3430237	total: 1.19s	remaining: 1.14s
510:	learn: 0.3429221	total: 1.19s	remaining: 1.14s
511:	learn: 0.3427256	total: 1.19s	remaining: 1.13s
512:	learn: 0.3426383	total: 1.19s	remaining: 1.13s
513:	learn: 0.3425833	total: 1.2s	remaining: 1.13s
514:	learn: 0.3425556	total: 1.2s	remaining: 1.13s
515:	learn: 0.3422990	total: 1.2s	remaining: 1.13s
516:	learn: 0.3422321	total: 1.2s	remaining: 1.12s
517:	learn: 0.3421823	total: 1.2s	remaining: 1.12s
518:	learn: 0.3418491	total: 1.21s	remaining: 1.12s
519:	learn: 0.3416781	total: 1.21s	remaining: 1.12s
520:	learn: 0.3416353	total: 1.21s	remaining: 1.11s
521:	learn: 0.3413057	total: 1.21s	remaining: 1.11s
522:	learn: 0.3412808	total: 1.22s	remaining: 1.11s
523:	learn: 0.3412169	total: 1.22s	remaining: 1.11s
524:	learn: 0.3410257	total: 1.22s	remaining: 1.1s
525:	learn: 0.3409

708:	learn: 0.3216824	total: 1.64s	remaining: 672ms
709:	learn: 0.3215562	total: 1.64s	remaining: 670ms
710:	learn: 0.3215286	total: 1.64s	remaining: 668ms
711:	learn: 0.3214762	total: 1.64s	remaining: 665ms
712:	learn: 0.3214254	total: 1.65s	remaining: 663ms
713:	learn: 0.3213739	total: 1.65s	remaining: 661ms
714:	learn: 0.3212946	total: 1.65s	remaining: 659ms
715:	learn: 0.3212551	total: 1.65s	remaining: 656ms
716:	learn: 0.3210965	total: 1.66s	remaining: 654ms
717:	learn: 0.3210763	total: 1.66s	remaining: 652ms
718:	learn: 0.3210013	total: 1.66s	remaining: 650ms
719:	learn: 0.3208575	total: 1.67s	remaining: 648ms
720:	learn: 0.3208401	total: 1.67s	remaining: 646ms
721:	learn: 0.3208169	total: 1.67s	remaining: 644ms
722:	learn: 0.3207495	total: 1.67s	remaining: 641ms
723:	learn: 0.3205190	total: 1.68s	remaining: 639ms
724:	learn: 0.3203717	total: 1.68s	remaining: 637ms
725:	learn: 0.3201857	total: 1.68s	remaining: 635ms
726:	learn: 0.3200694	total: 1.68s	remaining: 633ms
727:	learn: 

...

126:	learn: 0.4287214	total: 358ms	remaining: 2.46s
127:	learn: 0.4279456	total: 361ms	remaining: 2.46s
128:	learn: 0.4272530	total: 365ms	remaining: 2.47s
129:	learn: 0.4268545	total: 367ms	remaining: 2.46s
130:	learn: 0.4259613	total: 370ms	remaining: 2.45s
131:	learn: 0.4252709	total: 373ms	remaining: 2.46s
132:	learn: 0.4249394	total: 376ms	remaining: 2.45s
133:	learn: 0.4242581	total: 379ms	remaining: 2.45s
134:	learn: 0.4233835	total: 389ms	remaining: 2.49s
135:	learn: 0.4225679	total: 400ms	remaining: 2.54s
136:	learn: 0.4219773	total: 404ms	remaining: 2.54s
137:	learn: 0.4212240	total: 406ms	remaining: 2.54s
138:	learn: 0.4208694	total: 409ms	remaining: 2.53s
139:	learn: 0.4203034	total: 412ms	remaining: 2.53s
140:	learn: 0.4197989	total: 414ms	remaining: 2.52s
141:	learn: 0.4192886	total: 417ms	remaining: 2.52s
142:	learn: 0.4186604	total: 420ms	remaining: 2.52s
143:	learn: 0.4177568	total: 423ms	remaining: 2.51s
144:	learn: 0.4171348	total: 426ms	remaining: 2.51s
145:	learn: 

293:	learn: 0.3727897	total: 848ms	remaining: 2.04s
294:	learn: 0.3726493	total: 851ms	remaining: 2.03s
295:	learn: 0.3724402	total: 853ms	remaining: 2.03s
296:	learn: 0.3723169	total: 856ms	remaining: 2.02s
297:	learn: 0.3721114	total: 858ms	remaining: 2.02s
298:	learn: 0.3717804	total: 861ms	remaining: 2.02s
299:	learn: 0.3716989	total: 863ms	remaining: 2.01s
300:	learn: 0.3715407	total: 866ms	remaining: 2.01s
301:	learn: 0.3714145	total: 868ms	remaining: 2.01s
302:	learn: 0.3712681	total: 870ms	remaining: 2s
303:	learn: 0.3710532	total: 873ms	remaining: 2s
304:	learn: 0.3708023	total: 875ms	remaining: 1.99s
305:	learn: 0.3706063	total: 877ms	remaining: 1.99s
306:	learn: 0.3704497	total: 880ms	remaining: 1.99s
307:	learn: 0.3703298	total: 882ms	remaining: 1.98s
308:	learn: 0.3701517	total: 884ms	remaining: 1.98s
309:	learn: 0.3700588	total: 886ms	remaining: 1.97s
310:	learn: 0.3698348	total: 888ms	remaining: 1.97s
311:	learn: 0.3696610	total: 891ms	remaining: 1.96s
312:	learn: 0.3695

525:	learn: 0.3389481	total: 1.36s	remaining: 1.23s
526:	learn: 0.3387418	total: 1.37s	remaining: 1.23s
527:	learn: 0.3385941	total: 1.37s	remaining: 1.23s
528:	learn: 0.3384638	total: 1.37s	remaining: 1.22s
529:	learn: 0.3383989	total: 1.38s	remaining: 1.22s
530:	learn: 0.3382253	total: 1.38s	remaining: 1.22s
531:	learn: 0.3381715	total: 1.38s	remaining: 1.21s
532:	learn: 0.3378752	total: 1.38s	remaining: 1.21s
533:	learn: 0.3377747	total: 1.39s	remaining: 1.21s
534:	learn: 0.3376789	total: 1.39s	remaining: 1.21s
535:	learn: 0.3375278	total: 1.39s	remaining: 1.2s
536:	learn: 0.3373241	total: 1.39s	remaining: 1.2s
537:	learn: 0.3370073	total: 1.39s	remaining: 1.2s
538:	learn: 0.3369438	total: 1.4s	remaining: 1.19s
539:	learn: 0.3367811	total: 1.4s	remaining: 1.19s
540:	learn: 0.3367129	total: 1.4s	remaining: 1.19s
541:	learn: 0.3366310	total: 1.4s	remaining: 1.19s
542:	learn: 0.3365823	total: 1.41s	remaining: 1.18s
543:	learn: 0.3363219	total: 1.41s	remaining: 1.18s
544:	learn: 0.33623

744:	learn: 0.3167604	total: 1.88s	remaining: 642ms
745:	learn: 0.3165994	total: 1.88s	remaining: 639ms
746:	learn: 0.3164203	total: 1.88s	remaining: 637ms
747:	learn: 0.3163558	total: 1.88s	remaining: 634ms
748:	learn: 0.3162280	total: 1.88s	remaining: 632ms
749:	learn: 0.3161238	total: 1.89s	remaining: 629ms
750:	learn: 0.3159416	total: 1.89s	remaining: 627ms
751:	learn: 0.3157892	total: 1.89s	remaining: 624ms
752:	learn: 0.3156718	total: 1.9s	remaining: 622ms
753:	learn: 0.3155654	total: 1.9s	remaining: 619ms
754:	learn: 0.3154521	total: 1.9s	remaining: 617ms
755:	learn: 0.3153815	total: 1.9s	remaining: 614ms
756:	learn: 0.3152760	total: 1.91s	remaining: 612ms
757:	learn: 0.3152458	total: 1.91s	remaining: 609ms
758:	learn: 0.3152050	total: 1.91s	remaining: 607ms
759:	learn: 0.3151479	total: 1.91s	remaining: 604ms
760:	learn: 0.3150933	total: 1.92s	remaining: 601ms
761:	learn: 0.3150926	total: 1.92s	remaining: 599ms
762:	learn: 0.3148658	total: 1.92s	remaining: 596ms
763:	learn: 0.31

963:	learn: 0.2942318	total: 2.39s	remaining: 89.2ms
964:	learn: 0.2941374	total: 2.39s	remaining: 86.7ms
965:	learn: 0.2941263	total: 2.39s	remaining: 84.2ms
966:	learn: 0.2937931	total: 2.4s	remaining: 81.8ms
967:	learn: 0.2935076	total: 2.4s	remaining: 79.3ms
968:	learn: 0.2934372	total: 2.4s	remaining: 76.8ms
969:	learn: 0.2934053	total: 2.4s	remaining: 74.3ms
970:	learn: 0.2933347	total: 2.4s	remaining: 71.8ms
971:	learn: 0.2932131	total: 2.41s	remaining: 69.4ms
972:	learn: 0.2930849	total: 2.41s	remaining: 66.9ms
973:	learn: 0.2930662	total: 2.41s	remaining: 64.4ms
974:	learn: 0.2930247	total: 2.41s	remaining: 61.9ms
975:	learn: 0.2929797	total: 2.42s	remaining: 59.4ms
976:	learn: 0.2929270	total: 2.42s	remaining: 56.9ms
977:	learn: 0.2926870	total: 2.42s	remaining: 54.5ms
978:	learn: 0.2925774	total: 2.42s	remaining: 52ms
979:	learn: 0.2925298	total: 2.42s	remaining: 49.5ms
980:	learn: 0.2922138	total: 2.43s	remaining: 47ms
981:	learn: 0.2921807	total: 2.43s	remaining: 44.6ms
98

### 4.11. Predict Test set (with unknown results)

In [53]:
y_pred = classifier.predict(X_pred)

In [54]:
print(y_pred)

[0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 1 0 0 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 1
 1 0 0 0 0 0 1 1 0 0 0 1 0 0 0 1 1 0 0 0 0 0 1 0 0 0 1 1 1 1 0 0 1 1 0 1 0
 1 0 0 1 0 1 1 0 0 0 0 0 1 1 1 1 0 0 1 0 0 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0
 1 1 1 1 0 0 1 0 1 1 0 1 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0
 0 0 1 0 0 0 0 0 1 1 1 0 1 1 1 0 0 0 0 0 1 1 0 0 0 0 0 1 1 0 1 1 0 0 1 0 1
 0 1 0 0 0 0 0 1 0 1 0 1 1 0 1 1 1 0 1 0 0 0 0 1 0 0 0 0 1 0 0 1 0 1 0 1 0
 1 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 1 1 1 1 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 1 0 0 0 1 1 0 1 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 1 0 0 0 1 0 0
 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 1 0 0 0 1 0 1 0 0 0 0 1 1 0 1 0 0 1 1 0
 0 1 0 0 1 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 1 1 0 0 1 0 1 0 0 1 0 1 0 0 0 0
 0 1 1 1 1 1 0 1 0 0 0]


### 4.12. Create result dataframe and print it to csv

In [55]:
print(len(y_pred))

418


In [56]:
result = pd.DataFrame({'PassengerId': test['PassengerId'], 'Survived': y_pred})
print(result)

     PassengerId  Survived
0            892         0
1            893         0
2            894         0
3            895         0
4            896         0
..           ...       ...
413         1305         0
414         1306         1
415         1307         0
416         1308         0
417         1309         0

[418 rows x 2 columns]


In [57]:
result.to_csv('../00_data/catboost.csv', index=False)